In [23]:
import pandas as pd
#from selenium import webdriver
import time
from bs4 import BeautifulSoup
from pprint import pprint
import requests

In [24]:
import MatchStats as ms

In [25]:
MATCH_LIST_URL = "https://www.winstonslab.com/events/event.php?id=86#matches"

In [26]:
response = requests.get(MATCH_LIST_URL)
html = BeautifulSoup(response.text, "lxml")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.winstonslab.com
DEBUG:urllib3.connectionpool:https://www.winstonslab.com:443 "GET /events/event.php?id=86 HTTP/1.1" 200 None


In [27]:
past_div = html.find("div", {"id": "past"})

In [28]:
table_body = past_div.find("table").find("tbody")

In [29]:
rows = table_body.findAll("tr")

## Starting here we are working with an individual match

In [30]:
test_row = rows[1]

In [31]:
match_link = test_row.td.a
match_link

<a href="/matches/match.php?id=2448" id="tzDate_187"> 02:00 - Feb 10, 2018 </a>

Boom! thats the link we want.

In [32]:
match_id = match_link.attrs["href"].split("=")[-1]

There's the match id

In [34]:
df_2403 = ms.get_detailed_match_data(2403)

DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:65180/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"extensions": [], "args": []}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"extensions": [], "args": []}}}
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request {'sessionId': '90fa0e396faba4ac12870ed3259ad63b', 'status': 0, 'value': {'acceptSslCerts': True, 'applicationCacheEnabled': False, 'browserConnectionEnabled': False, 'browserName': 'chrome', 'chrome': {'chromedriverVersion': '2.34.522932 (4140ab217e1ca1bec0c4b4d1b148f3361eb3a03e)', 'userDataDir': '/var/folders/d8/yl8q86rx13d1fydd9pwjdhc80000gn/T/.org.chromium.Chromium.SjhnX5'}, 'cssSelectorsEnabled': True, 'databaseEnabled': False, 'handlesAlerts': True, 'hasTouchScreen': False, 'javascriptEnabled': True, 'locationContextEnabled': True, 'mobileE

DEBUG:selenium.webdriver.remote.remote_connection:DELETE http://127.0.0.1:65180/session/90fa0e396faba4ac12870ed3259ad63b/window {"sessionId": "90fa0e396faba4ac12870ed3259ad63b"}
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request {'sessionId': '90fa0e396faba4ac12870ed3259ad63b', 'status': 0, 'value': None}


In [35]:
df_2403.head(10)

,Match ID,Player,Hero,Time,Rating,FWin%,PTK,K/10,D/10,U/10,...,UE,UOOF,FK,FD,Impact,Elim/10,dmg/10,heal/10,ObjTime/10,ObjKills/10
0,2403,JJoNak,Zenyatta,00:47:21,1276,50.91%,12.5%,4.86,4.44,4.01,...,13.8,10.53%,7%,11%,762,N/A,N/A,N/A,N/A,N/A
1,2403,Saebyeolbe,Tracer,00:32:17,1174,52.63%,28.15%,11.77,4.65,5.27,...,31.98,0%,11%,8%,1265,N/A,N/A,N/A,N/A,N/A
2,2403,Libero,Soldier: 76,00:07:17,1058,71.43%,27.59%,10.98,5.49,4.12,...,28.57,0%,14%,29%,635,N/A,N/A,N/A,N/A,N/A
3,2403,clockwork,Tracer,00:16:37,1055,30%,31.25%,9.03,6.62,5.42,...,-15.71,0%,10%,5%,1293,N/A,N/A,N/A,N/A,N/A
4,2403,Bani,Mercy,00:47:55,1041,49.12%,0.65%,0.21,5.01,3.97,...,0.88,0%,0%,4%,1403,N/A,N/A,N/A,N/A,N/A
5,2403,MekO,Roadhog,00:06:50,1039,28.57%,25%,7.32,7.32,2.93,...,-28.57,0%,14%,0%,1438,N/A,N/A,N/A,N/A,N/A
6,2403,Mano,Winston,00:21:30,1037,38.46%,18.42%,6.51,6.51,3.26,...,11.54,14.29%,0%,8%,1328,N/A,N/A,N/A,N/A,N/A
7,2403,Libero,Genji,00:16:21,1036,47.37%,20.9%,8.56,5.5,3.67,...,19.3,0%,11%,5%,935,N/A,N/A,N/A,N/A,N/A
8,2403,ArK,Mercy,00:47:23,1035,49.12%,0.56%,0.21,4.22,3.8,...,29.45,5.88%,0%,7%,653,N/A,N/A,N/A,N/A,N/A
9,2403,Saebyeolbe,Mccree,00:07:26,1002,60%,26.92%,9.42,4.04,2.69,...,40,50%,0%,10%,714,N/A,N/A,N/A,N/A,N/A


## Now let's try to loop through all of the matches

In [22]:
columns = df_2403.columns.values

NameError: name 'df_2403' is not defined

In [27]:
base_df = pd.DataFrame(columns=columns, index=[])

In [32]:
for row in rows:
    match_link = row.td.a
    match_id = match_link.attrs["href"].split("=")[-1]
    match_df = ms.detailed_match_data(match_id)
    base_df = base_df.append(match_df, ignore_index=True)
    print("Finished Match: {}".format(match_id))

Finished Match: 2404
Finished Match: 2403
Finished Match: 2402
Finished Match: 2401
Finished Match: 2400
Finished Match: 2399
Finished Match: 2383
Finished Match: 2382
Finished Match: 2381
Finished Match: 2380
Finished Match: 2379
Finished Match: 2375


In [33]:
base_df.shape

(694, 22)

In [34]:
base_df.head(10)

,Match ID,Player,Hero,Time,Rating,FWin%,PTK,K/10,D/10,U/10,...,UE,UOOF,FK,FD,Impact,Elim/10,dmg/10,heal/10,ObjTime/10,ObjKills/10
0,2404,Fleta,Widowmaker,00:22:53,1476,61.54%,39.78%,16.17,4.37,4.81,...,18.46,9.09%,58%,0%,1565,N/A,N/A,N/A,N/A,N/A
1,2404,Wekeed,Junkrat,00:17:40,1229,68.18%,32.56%,15.85,6.79,3.96,...,31.82,0%,27%,5%,1479,N/A,N/A,N/A,N/A,N/A
2,2404,Surefour,Tracer,00:05:40,1218,62.5%,27.78%,8.82,5.29,7.06,...,4.17,0%,13%,13%,1800,N/A,N/A,N/A,N/A,N/A
3,2404,ryujehong,Zenyatta,00:31:30,1177,62.16%,12.06%,5.4,5.71,3.49,...,-6.61,10%,3%,14%,824,N/A,N/A,N/A,N/A,N/A
4,2404,zunba,D.Va,00:44:42,1128,58.93%,20.4%,9.17,5.37,0,...,0,0%,5%,0%,859,N/A,N/A,N/A,N/A,N/A
5,2404,Fleta,Reaper,00:06:34,1118,66.67%,35.71%,15.23,7.61,4.57,...,0,0%,50%,17%,1118,N/A,N/A,N/A,N/A,N/A
6,2404,BigG00se,Lucio,00:16:04,1104,35%,10.34%,3.73,5.6,3.11,...,15,0%,5%,0%,1020,N/A,N/A,N/A,N/A,N/A
7,2404,tobi,Mercy,00:43:37,1056,59.62%,0.53%,0.23,4.13,3.9,...,16.86,5.88%,0%,4%,1464,N/A,N/A,N/A,N/A,N/A
8,2404,KuKi,Winston,00:43:33,1039,58.18%,13.85%,6.2,6.2,3.9,...,4.32,5.88%,0%,5%,1095,N/A,N/A,N/A,N/A,N/A
9,2404,Surefour,Pharah,00:05:04,1036,28.57%,26.32%,9.87,5.92,1.97,...,0,0%,0%,14%,1232,N/A,N/A,N/A,N/A,N/A


In [35]:
base_df.tail(10)

,Match ID,Player,Hero,Time,Rating,FWin%,PTK,K/10,D/10,U/10,...,UE,UOOF,FK,FD,Impact,Elim/10,dmg/10,heal/10,ObjTime/10,ObjKills/10
684,2375,Kariv,Soldier: 76,00:00:03,NED,0%,0%,0,0,0,...,0,0%,0%,0%,996,N/A,N/A,N/A,N/A,N/A
685,2375,Sleepy,Ana,00:01:42,NED,50%,0%,0,5.88,5.88,...,-50,0%,0%,0%,1135,N/A,N/A,N/A,N/A,N/A
686,2375,Sleepy,Moira,00:00:41,NED,0%,0%,0,29.27,0,...,0,0%,0%,0%,1024,N/A,N/A,N/A,N/A,N/A
687,2375,Sleepy,Zarya,00:00:10,NED,0%,0%,0,0,0,...,0,0%,0%,0%,995,N/A,N/A,N/A,N/A,N/A
688,2375,Danteh,Genji,00:02:33,NED,50%,9.09%,3.92,7.84,3.92,...,50,0%,25%,0%,1366,N/A,N/A,N/A,N/A,N/A
689,2375,silkthread,Genji,00:03:17,NED,75%,19.05%,12.18,3.05,6.09,...,25,0%,0%,0%,1408,N/A,N/A,N/A,N/A,N/A
690,2375,silkthread,Mccree,00:02:31,NED,0%,50%,3.97,15.89,3.97,...,100,0%,0%,33%,311,N/A,N/A,N/A,N/A,N/A
691,2375,silkthread,Pharah,00:03:27,NED,75%,27.27%,8.7,2.9,5.8,...,-25,0%,0%,0%,1094,N/A,N/A,N/A,N/A,N/A
692,2375,silkthread,Tracer,00:02:53,NED,100%,12.5%,6.94,3.47,10.4,...,0,50%,0%,0%,672,N/A,N/A,N/A,N/A,N/A
693,2375,Envy,Bastion,00:00:50,NED,66.67%,0%,0,0,0,...,0,0%,0%,0%,986,N/A,N/A,N/A,N/A,N/A


In [36]:
base_df.to_csv("detailed_stats.csv")

In [38]:
base_df.columns.values

array(['Match ID', 'Player', 'Hero', 'Time', 'Rating', 'FWin%', 'PTK',
       'K/10', 'D/10', 'U/10', 'TTCU', 'KPU', 'UE', 'UOOF', 'FK', 'FD',
       'Impact', 'Elim/10', 'dmg/10', 'heal/10', 'ObjTime/10',
       'ObjKills/10'], dtype=object)